# Alexandria Curso Basico - MODULO 3

## Cargar datos
A continuacion se pueden visualizar multiples formas de cargars datos para su posterior procesamiento. Dentro del entorno python la libreria mas utilizada para este prorposito es pandas.

Esta herramienta nos permite cargar datos de diferentes tipos de fuentes de datos, los mismos pueden ser CSV, TSV, Sql, Bases de datos tradicionales, JSON, HTML, Haddop, SAS, Big query, Stata entre otros

!pip install --upgrade pandas-profiling

!pip install -U seaborn

In [ ]:
import numpy as np
import pandas as pd
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
df = pd.read_csv('./titanic.csv')

In [18]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Estética y luego transformación
## Agrupando columnas por tipo de datos

In [ ]:
tipos = df.columns.to_series().groupby(df.dtypes).groups
ctext = tipos[np.dtype('object')]
ctext

In [ ]:
columnas = df.columns  # lista de todas las columnas
cnum = list(set(columnas) - set(ctext))
cnum

## Imputación de nulos
### Datos cuantititavos

In [ ]:
for c in cnum:
    mean = df[c].mean()
    if df[c].isna().sum() > 0:
        df[c+'_nan'] = df[c].isna()
    df[c] = df[c].fillna(mean)

### Datos categóricos

In [ ]:
for c in ctext:
    mode = df[c].mode()[0]
    if df[c].isna().sum() > 0:
        df[c+'_nan'] = df[c].isna()
    df[c] = df[c].fillna(mode)

Y nuestro set de datos se verá de la siguiente manera...

In [ ]:
# VER ARCHIVO GENERADO PARA EL ANALISIS UNIVARIADO
profile = pandas_profiling.ProfileReport(df)
profile.to_file(outputfile="analisisUnivariado.html")

In [ ]:
df.head(10)

# MÓDULO 3
La tarea en este momento es identificar cuales son las variables que tienen que tipo de distribución y proponer que algún tipo de transformación

## Relación sobrevivientes con categoria y sexo
Hay relación?

In [ ]:
sns.heatmap(df.corr(),annot=True)

In [ ]:
agrupado = pd.crosstab(df.Pclass, df.Sex,values=df.Survived, aggfunc=np.mean)
sns.heatmap(agrupado,annot=True, cmap='RdYlGn')

## Relación sobrevivientes con categoria y edad 

In [ ]:
# Distribucion sobrevivientes por edad y por clase
agrupado = pd.crosstab(df.Pclass,pd.qcut(df.Age, 5),values=df.Survived, aggfunc=np.mean)

In [ ]:
agrupado

In [ ]:
# Distribucion sobrevivientes por edad y por clase
sns.heatmap(agrupado,annot=True, cmap='RdYlGn')

### Reorganizo la base

In [ ]:
# Agrupando columnas por tipo de datos
tipos = df.columns.to_series().groupby(df.dtypes).groups

# Armando lista de columnas categóricas
ctext = tipos[np.dtype('object')]
ctext

In [ ]:
# Armando lista de columnas numéricas
columnas = df.columns  # lista de todas las columnas
cnum = list(set(columnas) - set(ctext))
cnum

In [ ]:
# Controlando que no hayan valores faltantes
df.isnull().any().any()

# Split para análisis

In [ ]:
#split dataset in features and target variable
X = df.select_dtypes(include=['float64','int']).drop('Survived',axis=1) # Features
y = df.select_dtypes(include=['float64','int']).Survived # Target variable

In [ ]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [ ]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))